In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
# ImageDataGenerator 설정
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 검증 데이터셋 비율
)

# train_generator 정의
train_generator = datagen.flow_from_directory(
    'categorized_images',  # 데이터셋 폴더 경로 지정
    target_size=(150, 150),  # 이미지 크기
    batch_size=32,
    class_mode='categorical',  # 다중 분류
    subset='training'  # 트레이닝 데이터셋
)


# validation_generator 정의
validation_generator = datagen.flow_from_directory(
    'categorized_images',  # 데이터셋 폴더 경로 지정
    target_size=(150, 150),  # 이미지 크기
    batch_size=32,
    class_mode='categorical',  # 다중 분류
    subset='validation'  # 검증 데이터셋
)


Found 85 images belonging to 11 classes.
Found 15 images belonging to 11 classes.


[]


In [4]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam


# VGG16 모델 불러오기
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# 모델의 상위 레이어를 동결
for layer in base_model.layers:
    layer.trainable = False

# 커스텀 레이어 추가
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# 최종 모델
model = Model(inputs=base_model.input, outputs=predictions)

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10  # 에폭 수
)


Epoch 1/10
2/2 [==============================] - 3s 1s/step - loss: 2.2875 - accuracy: 0.2453
Epoch 2/10
2/2 [==============================] - 2s 1s/step - loss: 2.1476 - accuracy: 0.3396
Epoch 3/10
2/2 [==============================] - 3s 1s/step - loss: 2.0703 - accuracy: 0.3125
Epoch 4/10
2/2 [==============================] - 3s 1s/step - loss: 1.8174 - accuracy: 0.3594
Epoch 5/10
2/2 [==============================] - 3s 1s/step - loss: 1.6806 - accuracy: 0.4375
Epoch 6/10
2/2 [==============================] - 2s 1s/step - loss: 1.7434 - accuracy: 0.5283
Epoch 7/10
2/2 [==============================] - 3s 1s/step - loss: 1.7354 - accuracy: 0.4375
Epoch 8/10
2/2 [==============================] - 2s 905ms/step - loss: 1.5363 - accuracy: 0.4717
Epoch 9/10
2/2 [==============================] - 2s 1s/step - loss: 1.5425 - accuracy: 0.5094
Epoch 10/10
2/2 [==============================] - 3s 1s/step - loss: 1.3924 - accuracy: 0.5312


In [5]:

# 모델 저장
model.save('vgg16_model.h5')  # 모델을 저장할 파일 이름


c:\Users\gjaischool\anaconda3\envs\MetaWave\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
from tensorflow.keras.models import load_model

loaded_model = load_model('vgg16_model.h5')

c:\Users\gjaischool\anaconda3\envs\MetaWave\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\gjaischool\anaconda3\envs\MetaWave\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\gjaischool\anaconda3\envs\MetaWave\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\gjaischool\anaconda3\envs\MetaWave\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.2.0)/charset_normalizer (None) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [13]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

# 모델 파일 경로 설정
model_path = 'vgg16_model.h5'

# 모델 로드
model = load_model(model_path)

def classify_image(img_path):
    # 이미지 로드 및 크기 조정
    img = image.load_img(img_path, target_size=(150, 150))

    # 이미지를 배열로 변환
    img_array = image.img_to_array(img)

    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # 이미지 예측
    predictions = model.predict(img_array)

    # 가장 높은 확률을 가진 클래스 반환
    predicted_class = np.argmax(predictions, axis=1)
    return predicted_class

img_path = r'C:\Users\gjaischool\Desktop\MetaWave\MetaWave\test_img\cda25cac-5238-4380-b15e-84dcd07b1e06.png'
result = classify_image(img_path)

1/1 [==============================] - 0s 215ms/step


In [14]:
print("Predicted Class:", result)

Predicted Class: [8]


In [15]:
class_labels = []

# 폴더 경로 설정
folder_path = r'C:\Users\gjaischool\Desktop\MetaWave\MetaWave\categorized_images'


# 폴더 내의 모든 하위 폴더를 순회하여 클래스 레이블 생성
for folder_name in os.listdir(folder_path):
    if os.path.isdir(os.path.join(folder_path, folder_name)):
        class_labels.append(folder_name)

In [16]:
print(class_labels[result.max() -1])

love
